# NSU, vaje 2: meta učenje

### A: Spletna stran OpenML in knjižnica **openml**, priprava podatkov

A.1 Dobi podatke z OpenML-ja (https://www.openml.org/).
Najdi vsa podatkovja, ki imajo med 100 in 200 primerov in med 4 do 100 znacilk. Pomagaj si s funkcijo **openml.datasets.list_datasets**. Podaj ji argumente
- number_instances (npr. number_instances="\<x>..\<y>", kjer sta x in y najmanjse in najvecje dovoljeno stevilo primerov, npr. "10..20")
- number_features (analogno number_instances)
- output_format="dataframe"
in si oglej dobljeni pandasov Dataframe. Eden od stolpcev v njem je did (data ID).

Za opis OpenML funkcij lahko uporabis dokumentacijo na https://docs.openml.org/Python-API/

In [1]:
import numpy as np
import pandas as pd
import openml as oml

In [2]:
d = oml.datasets.list_datasets(number_instances='100..200', number_features='4..100', output_format="dataframe")
data_ids = list(d["did"])
print(d)

         did                 name  version uploader  status format  \
10        10                lymph        1        1  active   ARFF   
48        48                  tae        1        1  active   ARFF   
55        55            hepatitis        1        1  active   ARFF   
61        61                 iris        1        1  active   ARFF   
62        62                  zoo        1        1  active   ARFF   
...      ...                  ...      ...      ...     ...    ...   
43839  43839  IRIS-flower-dataset        1    30125  active   arff   
43859  43859            iriiiiiis        2    28351  active   ARFF   
44151  44151                 Iris       51    30495  active   arff   
44154  44154      iris_reproduced        1    30495  active   arff   
44344  44344                 Iris       52    30495  active   arff   

       MajorityClassSize  MaxNominalAttDistinctValues  MinorityClassSize  \
10                  81.0                          8.0                2.0   
48     

A.2 Nalozi ta podatkovja s funkcijo **openml.datasets.get_datasets**, tako da uporabis ID-je podatkovij iz prejsnje tocke. To bo prvic morda trajalo nekaj minut. Ker se bodo nalozena podatkovja shranila tudi na disk racunalnika, bo vsak naslednji klic precej hitrejsi.

In [3]:
podatkovja_vsa = oml.datasets.get_datasets(data_ids)
print(podatkovja_vsa)

[OpenML Dataset
Name..........: lymph
Version.......: 1
Format........: ARFF
Upload Date...: 2014-04-06 23:19:52
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/10/lymph.arff
OpenML URL....: https://www.openml.org/d/10
# of features.: 19
# of instances: 148, OpenML Dataset
Name..........: tae
Version.......: 1
Format........: ARFF
Upload Date...: 2014-04-06 23:22:53
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/48/tae.arff
OpenML URL....: https://www.openml.org/d/48
# of features.: 6
# of instances: 151, OpenML Dataset
Name..........: hepatitis
Version.......: 1
Format........: ARFF
Upload Date...: 2014-04-06 23:23:13
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/55/hepatitis.arff
OpenML URL....: https://www.openml.org/d/55
# of features.: 20
# of instances: 155, OpenML Dataset
Name..........: iris
Version.......: 1
Format........: ARFF
Upload Date...: 2014-04-06 23:23:39
Licence.......: Pu

A.3 Iz zbirke podatkovij odstrani tista, ki niso primerna za klasifikacijo. Pomagas si lahko s klicem **podatkovje.get_data()**, ki vrne

- x: pandasov Dataframe znacilk, ki vkljucuje tudi ciljno spremenljivko, ce argument target in podan
- y: stolpec, ki podaja vrednosti ciljne spremenljivke, ce je argument target podan, in None sicer
- nominalni: seznam vrednosti True/False, ki pove, ali je i-ti atribut nominalen
- atributi: seznam imen atributov

Ciljno spremenljivko posameznega podatkovja najdemo z **podatkovje.default_target_attribute**.

Iz vseh podatkovij iz prejsnje naloge odstrani tista, ki imajo
- neznano (None) ali numericno ciljno spremenljivko,
- več ciljnih spremenljivk.

Pazi, da obdržiš le eno razlicico podatkov: "iris" se npr. pojavi vec kot 40-krat. Ime podatkovja je shranjeno v
polju "name" (do njega torej dostopamo s "podatkovje.name")
Ker scikit ne podpira nominalnih znacilk, odstrani tudi vsa podatkovja, ki vsebujejo nominalne znacilke.


In [ ]:
podatkovja_ok = {}
for n, podatkovje in enumerate(podatkovja_vsa):
    ime_podatkovja = podatkovje.name
    target = podatkovje.default_target_attribute
    X, _, nominal, names = podatkovje.get_data()
    # Vec pogojev, da podatkovje obdrzimo:

    # ce ga nismo ze prej (tj. neke druge verzije)
    name_ok = ime_podatkovja not in podatkovja_ok

    # ce ima znan target in je target en sam (in ne npr. "Spol,Starost")
    targ_ok = target is not None and "," not in target

    # ce je target nominalen (klasifikacija) in podatki nimajo nominalnih atributov
    i_target = names.index(target) if targ_ok else 0
    nomi_ok = nominal[i_target] and sum(nominal) == 1  # natanko en nominalen in to je target
    if name_ok and targ_ok and nomi_ok:
        podatkovja_ok[ime_podatkovja] = n

n_vsa = len(podatkovja_vsa)
n_ok = len(podatkovja_ok)
print(f"Obdrzal sem {n_ok} podatkovij ({100 * n_ok / n_vsa:.1f}%)")


### B - Priprava ciljnih spremenljivk za meta učenje 
Naša ciljna spremenljivka bo uspešnost posameznih metod strojnega učenja na različnih podatkih. 

Na vseh OK podatkovjih pozeni modele kNN (**sklearn.neighbors.KNeighborsClassifier**), odlocitveno drevo (**sklearn.tree.DecisionTreeClassifier**) in naivnega Bayesa (**sklearn.naive_bayes.GaussianNB**).
Za smiselno oceno kvalitete je treba razbiti podakovje na ucno in testno mnozico. V testno mnozico daj 1/4 primerov. 
Kvaliteto napovedi izmeri s tocnostjo.
Ce se bo pri ucenju/napovedovanju koda sesula, to morda pomeni, da s podatki ni vse v redu in bi jih bilo potrebno prej
odfiltrirati.
Dobljene rezultate shrani v tabelo s stolpci

**ime podatkovja,knn,drevo,bayes,najboljsi**

kjer stolpci 2-4 podajajo tocnosti modelov na tesni podmnozici podatkovja, stolpec najboljsi pa ime najboljsega
modela. Pandasov dataframe lahko preprosto shraniš v CSV datoteko s funkcijo **df.to_csv**.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


knn = KNeighborsClassifier(n_neighbors=5)
tree = DecisionTreeClassifier()
bayes = GaussianNB()

modeli = {"knn": knn, "tree": tree, "bayes": bayes}
data_dict = {"name": [], "knn": [], "tree": [], "bayes": [], "best": []}

for ime_podatkovja, n in podatkovja_ok.items():
    x, y, nominal, names = podatkovja_vsa[n].get_data(target=podatkovja_vsa[n].default_target_attribute)
    x = np.array(x)
    y = np.array([str(t) for t in y])  # Ker je prej y tu in tam vseboval True in False

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2023)

    data_dict["name"].append(ime_podatkovja)
    best_acc, best_model = 0, ""
    
    for model_name, model in modeli.items():
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        accuracy = accuracy_score(y_test, predictions)
        if accuracy > best_acc:
            best_model = model_name
            best_acc = accuracy
        data_dict[model_name].append(accuracy)
    data_dict["best"].append(best_model)

meta_df = pd.DataFrame(data=data_dict, index=None)
meta_df.to_csv("meta_target.csv", index=False)


### C Priprava napovednih spremenljivk za meta učenje

Pri pripravi meta znacilk nam bo pomagal paket pymfe https://pypi.org/project/pymfe/.
Najbolj koristno bo orodje **MFE**, ki mu v argumentu **groups** povemo, katere tipe meta značilk želimo.
Po nastavitvi moramo poklicati še njegovi funkciji **fit** in **extract**.

Najprej preizkusimo pymfe na enem izmed podatkovij v naši zbirki.

C.1 Naredi meta značilke iz podatkov (tip značilk "general" in "info-theory"). Oglej si ustvarjene značilke. 
Preizkusiš lahko tudi druge možnosti, npr. "statistical".

Opozorilo, da nekaterih značilk ni mogoče izračunati, ni nič nenavadnega.

In [ ]:
from pymfe.mfe import MFE

n = list(podatkovja_ok.values())[0]
x, y, nominal, names = podatkovja_vsa[n].get_data(target=podatkovja_vsa[n].default_target_attribute)
x = np.array(x)
y = np.array([str(t) for t in y])

mfe1 = MFE(groups=["general", "info-theory"])
mfe1.fit(x, y)
attribute_names, attribute_values = mfe1.extract()

print(list(zip(attribute_names, attribute_values)))

C.2 Naredi meta značilke na podlagi enostavnega modela - odločitvenega drevesa, naučenega na celotni množici. Model boš moral ustvariti in naučiti sam, značilke pa iz naučenega drevesa pridobiš z **MFE** in nastavitvijo "model-based" ter funkcjo **extract_from_model**.

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(x, y)
mfe2 = MFE(groups=["model-based"])
attribute_names2, attribute_values2 = mfe2.extract_from_model(tree)

print(list(zip(attribute_names2, attribute_values2)))

C.3 Izračunaj vse tri vrste značilk za vsa podatkovja v naši zbirki in jih shrani v tabelo. Naredi tabelo s stolpci

**name,znacilka1,znacilka2,...,znacilkaN**, 

kjer je N stevilo znacilk (imena stolpcev 2-N niso znacilkaI, ampak dejanska imena znacilk, ki se ustvarijo.)
Tabela lahko vsebuje manjkajoce vrednosti.

In [ ]:
meta_features = {"name": []}
for ime_podatkovja, n in podatkovja_ok.items():
    x, y, nominal, names = podatkovja_vsa[n].get_data(target=podatkovja_vsa[n].default_target_attribute)
    x = np.array(x)
    y = np.array([str(t) for t in y])  # Ker je prej y tu in tam vseboval True in False

    mfe1 = MFE(groups=["general", "info-theory"]) 
    mfe2 = MFE(groups=["model-based"])
    mfe1.fit(x, y)
    attribute_names, attribute_values = mfe1.extract()
    tree = DecisionTreeClassifier()
    tree.fit(x, y)
    attribute_names2, attribute_values2 = mfe2.extract_from_model(tree)
    attribute_names += attribute_names2
    attribute_values += attribute_values2
    
    is_first = len(meta_features) == 1
    meta_features["name"].append(ime_podatkovja)
    for a_name, a_value in zip(attribute_names, attribute_values):
        if is_first:
            meta_features[a_name] = [a_value]
        else:
            meta_features[a_name].append(a_value)
pd.DataFrame(data=meta_features, index=None).to_csv("meta_features.csv", index=False)
